In [8]:
import os
import re
import subprocess
def reversesentence(fns):
  writetexts = []
  word = ''
  textsnt = ''
  print("start to loading sentence")
  print(fns)
  textrnmls = open('/content/rnmlall/' + fns + '.rnml', 'r', encoding='utf-8')
  lnrnml = 0
  lnsnt = 0
  textsnts = open('/content/sntall/' + fns, 'r', encoding='utf-8')
  while True:
    textsntsplits = textsnts.readline()
    if not textsntsplits:
      break
#    print(textsntsplits.rstrip(os.linesep))
    textsntsplits = textsntsplits.rstrip(os.linesep)
    textrnml = textrnmls.readline().rstrip(os.linesep)
    for textsnt in textsntsplits.splitlines():
      if textsnt == '':
        continue
      if '<span>' not in textrnml:
        wordsnt = textcompare(textsnt, textrnml,[])
      else:
        textrnmlrmspan, spanrmstarts = rmspan(textrnml)
        wordsnt = textcompare(textsnt, textrnmlrmspan, spanrmstarts)
      if word == '':
        word = wordsnt
      else:
        word = word + "\n" + wordsnt
#      print(wordsnt.rstrip(os.linesep))
#  w = open('/content/' + fns + '.rsnt', 'w')
#  w.write(word)
#  w.close()
  textsnts.close()
  textrnmls.close()
#  print(word)
  return word

def textcompare(textsnt, textrnml, spanrmstarts):
#  print(textsnt)
  rnmi = 0
  snti = 0
  wordsnt = ''
  if spanrmstarts == []:
#    print('%s\n%s' % (textsnts, textrnml))
    for textsntchar in textsnt:
      if rnmi > len(textrnml) - 1:
        wordsnt += textsntchar
        rnmi += 1
        continue
#      print(textsntchar, textrnml[rnmi], rnmi)
      if textsntchar == 'つ' and textrnml[rnmi] == 'っ':
        wordsnt += 'つ'
        rnmi += 1
        #print(wordsnt,snti)
      elif textsntchar == textrnml[rnmi]:
        wordsnt += textsntchar
        rnmi += 1
        #print(wordsnt,snti)
      else:
        wordsnt += textsntchar
        #print(wordsnt)
  else:
#    print(textsnt)
#    print(textrnml)
#    print(spanrmstarts)
    spanstarts = [list(x) for x in zip(*spanrmstarts)][0]
    spanends = [list(x) for x in zip(*spanrmstarts)][1]
    kouho = [list(x) for x in zip(*spanrmstarts)][2]
    sentaku = [list(x) for x in zip(*spanrmstarts)][3]
#    print(spanstarts, spanends, kouho, sentaku)
    for textsntchar in textsnt:
      if rnmi > len(textrnml) - 1:
        wordsnt += textsntchar
        rnmi += 1
        continue
#      print(textsntchar, textrnml[rnmi], rnmi)
      if textsntchar == 'つ' and textrnml[rnmi] == 'っ':
        if rnmi in spanstarts:
          wordsnt += '<span>'
        wordsnt += 'つ'
        rnmi += 1
        if rnmi in spanends:
#        print(spanrmstarts[spanends.index(rnmi)][3])
          wordsnt += '{%s}</span>' % spanrmstarts[spanends.index(rnmi)][3]
      elif textsntchar == textrnml[rnmi]:
        if rnmi in spanstarts:
          wordsnt += '<span>'
        wordsnt += textsntchar
        rnmi += 1
        if rnmi in spanends:
#        print(spanrmstarts[spanends.index(rnmi)][3])
          wordsnt += '{%s}</span>' % spanrmstarts[spanends.index(rnmi)][3]
      else:
        wordsnt += textsntchar
#  print(wordsnt)
  return wordsnt

  #<span>除く{'あっ': 1}</span>。

def rmspan(textrnml):
  spanstarts = [m.start() for m in re.finditer('<span>', textrnml)]
  spanends = [m.start() for m in re.finditer('</span>', textrnml)]
#  print(textrnml)
#  print (spanstarts, spanends)
  textrnmlrmspan = textrnml[:spanstarts[0]]
  spanrmstarts = []
  for spani in range(0,len(spanstarts)):
    kouho, sentaku = textrnml[spanstarts[spani] + len('<span>'):spanends[spani] - 1].split('{')
#    sentaku = sentaku.replace("'","")
#    print(spani, kouho, sentaku)
    #spanrmstarts.append([len(textrnmlrmspan), len(textrnmlrmspan) + len(kouho), kouho, sentaku])
    rnmlspanstart = len(textrnmlrmspan)
    textrnmlrmspan += kouho
    rnmlspanend = len(textrnmlrmspan)
    if spani < len(spanstarts) - 1:
      textrnmlrmspan += textrnml[spanends[spani] + len('</span>'):spanstarts[spani + 1] - 1]
    elif len(textrnml) > spanends[-1] + len('</span>'):
      textrnmlrmspan += textrnml[spanends[-1] + len('</span>'):]
    spanrmstarts.append([rnmlspanstart, rnmlspanend, kouho, sentaku])
#  print(textrnmlrmspan)
#  print(spanrmstarts)
  return textrnmlrmspan, spanrmstarts

#f = open('/content/nmlall/323AC0000000025_20210301_501AC0000000071.sentence.rnml')
#fns = '323AC0000000025_20210301_501AC0000000071'
#bu = 24
#rnml = f.readlines()
#writetexts = reversexml(rnml, fns)
#writetexts = reversesentence(fns)
#textrstc = open('/content/' + fns + '.rsnt', 'w', encoding='utf-8')
#textrstc.write(writetexts)
#textrstc.close()

dictsentenceall = {}
%cd /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/01-50wakatiall/
lswakatiall = " ".join(['ls','*.wwakatiall'])
output = subprocess.getoutput(lswakatiall)
%cd /content/
for fni in output.split('\n'):
  if fni[0:2] in dictsentenceall:
    dictsentenceall[fni[0:2]].append(fni.strip())
  else:
    dictsentenceall.update({fni[0:2] : [fni.strip()]})
for bu in range(1, 51):
  if bu in [13, 36]:
    f=open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/sortuniq/bunruiwall-sort-uniq-%02d-m.csv' % bu)
  else:
    f=open('/content/drive/My Drive/Colab Notebooks/houreibunrui20210510/sortuniq/bunruiwall-sort-uniq-%02d.csv' % bu)
  lines=f.readlines()
  f.close()
  writetext = []
  writetexts = []
  outputs = []
  for line in lines:
    fns = line[3:52]
    bunrui = line[0:2]
    writetexts = reversesentence(fns)
    textrstc = open('/content/' + fns[:-9] + '.rsnt', 'w', encoding='utf-8')
    textrstc.write(writetexts)
    textrstc.close()
#fns = '325CO0000000089_20201212_502CO0000000282.sentence'
#bunrui = '06'
#writetexts = reversesentence(fns)
#textrstc = open('/content/' + fns[:-9] + '.rsnt', 'w', encoding='utf-8')
#textrstc.write(writetexts)
#textrstc.close()

#  ziprsnt = " ".join(['zip', '/content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/rsntall.zip', '*.rsnt'])
#  print(subprocess.getoutput(ziprsnt))
#  subprocess.getoutput('rm /content/*.rsnt')
#  with open('merged' + fns + '.txt', 'a') as fwt:
#    for ls in zip(*writetexts):
#      fwt.write('|'.join(ls) + "\n")
!zip /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/rsntall.zip *.rnml

#ziprsnt = " ".join(['zip', '/content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/rsnt%02d' % bu, '*.rstn'])
#print(subprocess.getoutput(ziprsnt))
#subprocess.getoutput('rm /content/*.rsnt')

ストリーミング出力は最後の 5000 行に切り捨てられました。
updating: 426M60000a08002_20190701_501M60000a08001.rsnt (deflated 63%)
updating: 427M60000001007_20161001_000000000000000.rsnt (deflated 66%)
updating: 427M60000002049_20200901_502M60000002058.rsnt (deflated 82%)
updating: 427M60000010040_20170922_429M60000010031.rsnt (deflated 66%)
updating: 427M60000012004_20190315_431M60000012001.rsnt (deflated 65%)
updating: 427M60000080029_20170401_999M60000080029.rsnt (deflated 73%)
updating: 427M60000110001_20200901_502M60000110006.rsnt (deflated 74%)
updating: 427M60000a08002_20161001_000000000000000.rsnt (deflated 62%)
updating: 427M60001002001_20200214_502M60001002001.rsnt (deflated 66%)
updating: 428M60000080002_20161001_000000000000000.rsnt (deflated 62%)
updating: 428M60000800066_20200929_502M60000800079.rsnt (deflated 76%)
updating: 428M60000800067_20161001_000000000000000.rsnt (deflated 51%)
updating: 430AC0000000037_20190401_000000000000000.rsnt (deflated 77%)
updating: 430CO0000000177_20190401_430CO00000

KeyboardInterrupt: ignored

In [1]:
!unzip -q /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/rclone-current-linux-amd64.zip -d /content/
%cd /content/rclone-v1.56.0-linux-amd64/
!cp /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/install.sh ./
!sudo bash install.sh
#!sudo apt install vim
#!cp /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/predall/*.* /content/
!unzip -q /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/predall.zip -d /content/
!unzip -q /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/nmlall.zip -d /content/
!mkdir /content/rnmlall
!unzip -q /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/rnmlall.zip -d /content/rnmlall/
!mkdir /content/xmlall
!unzip -q /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/xmlall.zip -d /content/xmlall/
!mkdir /content/sntall
!unzip -q /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/sntall.zip -d /content/sntall/
#!unzip /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/allposlistset.zip -d /content/
#!unzip /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/termvecpkl.zip -d /content/

/content/rclone-v1.56.0-linux-amd64


In [9]:
#!cp /content/drive/MyDrive/Colab\ Notebooks/rclone.conf /root/.config/rclone/
#!tmux source /content/drive/MyDrive/Colab\ Notebooks/.tmux.conf
#!sudo apt install vim
#!rclone copy onedrive:/modelzip01-26/ /content/
#!unzip /content/drive/MyDrive/Colab\ Notebooks/allposlistset.zip
#!unzip /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/wakatidataset.zip -d /content/
#!unzip /content/drive/MyDrive/Colab\ Notebooks/houreibunrui20210510/termvecpkl.zip -d /content/
#!zip /content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/allposlistset.zip *.allposlist.pkl
import os
import re
import subprocess
%cd /content/
#lswakatiall = " ".join(['unzip', '/content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/modelzip/models49-00w.wwakatiall.zip', '-d', '/content/'])
#output = subprocess.getoutput(lawakatiall)
for fni in range(1, 51):
  unzipsnt = " ".join(['unzip', '/content/drive/My\ Drive/Colab\ Notebooks/houreibunrui20210510/rsntzip/rsnt%02d.zip' % fni, '-d', '/content/']) 
  subprocess.getoutput(unzipsnt)

/content
